In [1]:
import numpy as np
import pandas as pd
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as utils

In [2]:
def trainingModel(dataLoader):
    for epoch in range(10):
        ave_loss=0.0
        print("Epoch ",epoch+1)
        for i,data in enumerate(dataLoader,0):
            inputs,target =data
            #inputs = inputs.float()
            target = target.float()
            optimizer.zero_grad()
            output=model(inputs)
            #output = output.long()
            target = target.unsqueeze(1)
            loss=criteria(output,target)
            loss.backward()
            optimizer.step()
            ave_loss+=loss.item()
        print("Loss for Epoch ",epoch+1," is ",(ave_loss/x_train.shape[0]))

In [3]:
#change the dataframe to numpy array to tensor array
def changeToTensor(dataFrame):
    dataFrame= (dataFrame-dataFrame.min())/(dataFrame.max()-dataFrame.min())
    x = dataFrame.values
    x= x.astype('float64')
    x= torch.Tensor(x)
    return x

In [4]:
train_data = pd.read_csv('train.txt',sep=',')
test_data = pd.read_csv('test.txt',sep=',')

In [5]:
train_data.head()

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
1,2015-02-04 17:51:00,23.18,27.2720,426.0,721.25,0.004793,1
2,2015-02-04 17:51:59,23.15,27.2675,429.5,714.00,0.004783,1
3,2015-02-04 17:53:00,23.15,27.2450,426.0,713.50,0.004779,1
4,2015-02-04 17:54:00,23.15,27.2000,426.0,708.25,0.004772,1
5,2015-02-04 17:55:00,23.10,27.2000,426.0,704.50,0.004757,1


In [6]:
### create a model with pytorch#####
class Net(nn.Module):
    def __init__(self,size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(size,2000)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(2000,1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [7]:
train_numpy = train_data.values
labels = train_numpy[:,-1]
labels= labels.astype('float64')
labels= torch.Tensor(labels)
train_sub = train_data.drop(['date','Occupancy'],axis=1)
x_train = changeToTensor(train_sub)

In [8]:
labels = labels.unsqueeze(1)

In [9]:
x_train.shape, labels.shape

(torch.Size([8143, 5]), torch.Size([8143, 1]))

In [10]:
model = Net(x_train.shape[1])

In [11]:
model

Net(
  (fc1): Linear(in_features=5, out_features=2000, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=2000, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [12]:
criteria= nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [13]:
def train_epoch(model, opt, criterion, batch_size=50):
    model.train()
    losses = []
    for beg_i in range(0, x_train.size(0), batch_size):
        x_batch = x_train[beg_i:beg_i + batch_size, :]
        y_batch = labels[beg_i:beg_i + batch_size, :]
#         x_batch = Variable(x_batch)
#         y_batch = Variable(y_batch)
        opt.zero_grad()
        y_hat = model(x_batch)
        loss = criterion(y_hat, y_batch)
        loss.backward()
        opt.step()        
        losses.append(loss.data.numpy())
    return losses
    

In [15]:
num_epochs = 200
for e in range(num_epochs):
    e_losses = train_epoch(model,optimizer ,criteria,x_train.shape[0])
    print("For epoch ",e+1,"loss is ",e_losses)

For epoch  1 loss is  [array(0.72885686, dtype=float32)]
For epoch  2 loss is  [array(0.6558166, dtype=float32)]
For epoch  3 loss is  [array(0.60309345, dtype=float32)]
For epoch  4 loss is  [array(0.5678936, dtype=float32)]
For epoch  5 loss is  [array(0.54588014, dtype=float32)]
For epoch  6 loss is  [array(0.5321321, dtype=float32)]
For epoch  7 loss is  [array(0.52231324, dtype=float32)]
For epoch  8 loss is  [array(0.51332825, dtype=float32)]
For epoch  9 loss is  [array(0.5033764, dtype=float32)]
For epoch  10 loss is  [array(0.49167514, dtype=float32)]
For epoch  11 loss is  [array(0.47814584, dtype=float32)]
For epoch  12 loss is  [array(0.46313557, dtype=float32)]
For epoch  13 loss is  [array(0.4472326, dtype=float32)]
For epoch  14 loss is  [array(0.43111473, dtype=float32)]
For epoch  15 loss is  [array(0.41545853, dtype=float32)]
For epoch  16 loss is  [array(0.40082586, dtype=float32)]
For epoch  17 loss is  [array(0.38759664, dtype=float32)]
For epoch  18 loss is  [arra

For epoch  143 loss is  [array(0.06275105, dtype=float32)]
For epoch  144 loss is  [array(0.06238598, dtype=float32)]
For epoch  145 loss is  [array(0.06202828, dtype=float32)]
For epoch  146 loss is  [array(0.06167765, dtype=float32)]
For epoch  147 loss is  [array(0.06133426, dtype=float32)]
For epoch  148 loss is  [array(0.06099817, dtype=float32)]
For epoch  149 loss is  [array(0.06066954, dtype=float32)]
For epoch  150 loss is  [array(0.06034875, dtype=float32)]
For epoch  151 loss is  [array(0.06003475, dtype=float32)]
For epoch  152 loss is  [array(0.05972778, dtype=float32)]
For epoch  153 loss is  [array(0.05942732, dtype=float32)]
For epoch  154 loss is  [array(0.05913348, dtype=float32)]
For epoch  155 loss is  [array(0.0588458, dtype=float32)]
For epoch  156 loss is  [array(0.05856417, dtype=float32)]
For epoch  157 loss is  [array(0.0582878, dtype=float32)]
For epoch  158 loss is  [array(0.05801702, dtype=float32)]
For epoch  159 loss is  [array(0.05775212, dtype=float32)]

In [16]:
test_numpy = test_data.values
output1 = test_numpy[:,-1]
output1=output1.astype('float64')
output1 = torch.Tensor(output1)
test_sub = test_data.drop(['date','Occupancy'],axis=1)
x_test = changeToTensor(test_sub)

In [34]:
model = model.eval()
with torch.no_grad():
    predictions = model(x_test)

In [35]:
predictions.shape

torch.Size([9752, 1])

In [36]:
test = np.asarray(predictions)

In [37]:
print(type(test))

<class 'numpy.ndarray'>


In [39]:
test[test>0.5]=1
test[test<=0.5]=0

In [40]:
output1 = test_numpy[:,-1]

In [41]:
test.shape, output1.shape

((9752, 1), (9752,))

In [42]:
count=0
for i in range(test.shape[0]):
    if(output1[i]==test[i]):
        count = count+1
print("Testing accuracy is ", (count/test)*100)

SyntaxError: invalid syntax (<ipython-input-42-069d9247d360>, line 2)